In [24]:
# 静岡県、山梨県、神奈川県の熊出没マップPDFを取得
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import requests
import os

# Seleniumの設定
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # ヘッドレスモードで実行
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome(options=options)

# 山梨県のURLにアクセス
yamanashi_url = "https://www.pref.yamanashi.jp/shizen/kuma2.html"
driver.get(yamanashi_url)

try:
    # 山梨県のPDFリンクを取得
    wait = WebDriverWait(driver, 10)
    link = wait.until(
        EC.presence_of_element_located((By.PARTIAL_LINK_TEXT, "令和6年度（2024年度）ツキノワグマ出没・目撃情報"))
    )
    pdf_url = link.get_attribute("href")
    print(f"山梨県のPDFのURLを取得しました: {pdf_url}")

    # requestsでPDFをダウンロード
    pdf_path = os.path.join(os.getcwd(), "kuma_r6_yamanashi.pdf")
    response = requests.get(pdf_url)
    with open(pdf_path, 'wb') as pdf_file:
        pdf_file.write(response.content)
    print(f"山梨県のPDFを保存しました: {pdf_path}")

except Exception as e:
    print(f"山梨県のエラーが発生しました: {str(e)}")

# 静岡県のURLにアクセス
shizuoka_url = "https://www.pref.shizuoka.jp/kurashikankyo/shizenkankyo/wild/1017680.html"
driver.get(shizuoka_url)

try:
    # 静岡県のPDFリンクを取得
    wait = WebDriverWait(driver, 10)
    link = wait.until(
        EC.presence_of_element_located((By.PARTIAL_LINK_TEXT, "【NEW】クマ出没マップ"))
    )
    pdf_url = link.get_attribute("href")
    print(f"静岡県のPDFのURLを取得しました: {pdf_url}")

    # requestsでPDFをダウンロード
    pdf_path = os.path.join(os.getcwd(), "kuma_r6_shizuoka.pdf")
    response = requests.get(pdf_url)
    with open(pdf_path, 'wb') as pdf_file:
        pdf_file.write(response.content)
    print(f"静岡県のPDFを保存しました: {pdf_path}")

except Exception as e:
    print(f"静岡県のエラーが発生しました: {str(e)}")

# 神奈川県のURLにアクセス
kanagawa_url = "https://www.pref.kanagawa.jp/docs/t4i/cnt/f3813/"
driver.get(kanagawa_url)

try:
    # 神奈川県のPDFリンクを取得
    wait = WebDriverWait(driver, 10)
    link = wait.until(
        EC.presence_of_element_located((By.PARTIAL_LINK_TEXT, "ツキノワグマの目撃等情報を更新しました"))
    )
    pdf_url = link.get_attribute("href")
    print(f"神奈川県のPDFのURLを取得しました: {pdf_url}")

    # requestsでPDFをダウンロード
    pdf_path = os.path.join(os.getcwd(), "kuma_r6_kanagawa.pdf")
    response = requests.get(pdf_url)
    with open(pdf_path, 'wb') as pdf_file:
        pdf_file.write(response.content)
    print(f"神奈川県のPDFを保存しました: {pdf_path}")

except Exception as e:
    print(f"神奈川県のエラーが発生しました: {str(e)}")

finally:
    driver.quit()


山梨県のPDFのURLを取得しました: https://www.pref.yamanashi.jp/documents/61009/20241225kumamokugeki.pdf
山梨県のPDFを保存しました: /Users/nozomukitamura/Berkeley/kuma_r6_yamanashi.pdf
静岡県のPDFのURLを取得しました: https://www.pref.shizuoka.jp/_res/projects/default_project/_page_/001/017/680/241218kuma.pdf
静岡県のPDFを保存しました: /Users/nozomukitamura/Berkeley/kuma_r6_shizuoka.pdf
神奈川県のPDFのURLを取得しました: https://www.pref.kanagawa.jp/documents/15077/kuma_r6_1223.pdf
神奈川県のPDFを保存しました: /Users/nozomukitamura/Berkeley/kuma_r6_kanagawa.pdf


In [25]:
#神奈川県のクマ目撃情報のPDFから情報を抽出してbear_sightings_kanagawaへ
import pdfplumber
import os
import json
import re
import pandas as pd

# 神奈川県のPDFからテキストを抽出
pdf_path = os.path.join(os.getcwd(), "kuma_r6_kanagawa.pdf")
json_path = os.path.join(os.getcwd(), "bear_sightings_kanagawa.json")

try:
    sightings = []
    all_lines = []
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text = page.extract_text()
            if text:
                lines = text.split('\n')
                all_lines.extend(lines)

    column_titles = ["月日", "時間", "頭数", "状況", "場所等", "区分", "目撃・痕跡", "その他"]
    date_pattern = re.compile(r'(1[0-2]|[1-9])月(\d{1,2})日')

    for line in all_lines:
        # タイトル・不要行除外
        if (not line.strip() or 
            '《目撃・痕跡・その他》' in line or 
            all(title in line for title in column_titles)):
            continue

        m = date_pattern.search(line)
        if not m:
            # 日付パターンが見つからない行はスキップ
            continue

        date_str = m.group(0)
        month_str = m.group(1)  # 月(例えば"11"や"1")
        
        # 日付文字列の開始位置
        date_start = m.start()
        
        # IDやその他データは日付より前後にある
        id_with_date_part = line[:date_start]  # IDと紛れ込んだ数字を含む部分
        after_date_part = line[m.end():].strip()  # 日付以降の文字列

        # ID抽出処理
        if len(month_str) == 2:
            # 二桁月の場合、ID末尾を1文字削る
            id_str = id_with_date_part[:-1]
        else:
            # 一桁月の場合はそのまま
            id_str = id_with_date_part

        # IDは数字のみのはず
        id_str = ''.join(ch for ch in id_str if ch.isdigit())

        # 後半部分をsplitして time, number_of_bears, status, location, area_type, observation_typeを割り当て
        parts = after_date_part.split()

        if len(parts) >= 5:
            area_type = parts[-2]
            observation_type = parts[-1]
            time = parts[0]
            number_of_bears = parts[1]
            status = parts[2]

            if len(parts) > 5:
                # 場所は余った真ん中部分
                location = ' '.join(parts[3:-2])
            else:
                location = ''

            sighting = {
                "id": id_str,
                "date": date_str,
                "time": time,
                "number_of_bears": number_of_bears,
                "status": status,
                "location": location,
                "area_type": area_type,
                "observation_type": observation_type
            }
            sightings.append(sighting)
        else:
            # カラム数が合わない場合はログを確認し、PDFフォーマットに合わせて調整
            print(f"Line parsing issue: {line}")
            print(f"After date parts: {parts}")

    # JSONに変換する直前に、全ての辞書から"id"キーを削除
    for s in sightings:
        if "id" in s:
            del s["id"]

    # JSONファイルに保存
    with open(json_path, 'w', encoding='utf-8') as f:
        json.dump(sightings, f, ensure_ascii=False, indent=2)
    print(f"データをJSONファイルに保存しました: {json_path}")

except Exception as e:
    print(f"エラーが発生しました: {str(e)}")


Line parsing issue: 令和６年12月23日時点
After date parts: ['時点']
Line parsing issue: 令和６年12月23日時点
After date parts: ['時点']
Line parsing issue: 令和６年12月23日時点
After date parts: ['時点']
Line parsing issue: 令和６年12月23日時点
After date parts: ['時点']
データをJSONファイルに保存しました: /Users/nozomukitamura/Berkeley/bear_sightings_kanagawa.json


In [26]:
import pdfplumber
import os
import json
import re

pdf_path = os.path.join(os.getcwd(), "kuma_r6_kanagawa.pdf")
json_path = os.path.join(os.getcwd(), "bear_sightings_kanagawa.json")

try:
    sightings = []
    all_lines = []
    
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text = page.extract_text()
            if text:
                lines = text.split('\n')
                all_lines.extend(lines)

    column_titles = ["月日", "時間", "頭数", "状況", "場所等", "区分", "目撃・痕跡", "その他"]
    date_pattern = re.compile(r'(1[0-2]|[1-9])月(\d{1,2})日')
    
    # 補足情報を示す文字列パターン
    note_patterns = [
        "有害鳥獣捕獲",
        "くくり罠",
        "箱罠",
        "放獣",
        "捕獲",
        "飼養",
    ]

    for line in all_lines:
        # タイトル・不要行除外
        if (not line.strip() or
            '《目撃・痕跡・その他》' in line or
            all(title in line for title in column_titles)):
            continue

        m = date_pattern.search(line)
        if not m:
            continue

        date_str = m.group(0)
        after_date_part = line[m.end():].strip()

        # 後半部分をsplitして各フィールドを割り当て
        parts = after_date_part.split()
        if len(parts) >= 5:
            time = parts[0]
            number_of_bears = parts[1]
            status = parts[2]
            area_type = parts[-2]
            observation_type = parts[-1]
            
            # 場所情報の処理
            location_parts = parts[3:-2]  # 場所等カラムの全ての要素を取得
            
            # 最初の部分を場所として扱う
            location = ""
            if location_parts:
                location = location_parts[0]
                
                # もし最初の部分が補足情報っぽい場合は、次の部分を確認
                if any(note in location for note in note_patterns) and len(location_parts) > 1:
                    location = location_parts[1]

            sighting = {
                "date": date_str,
                "time": time,
                "number_of_bears": number_of_bears,
                "status": status,
                "location": location,
                "area_type": area_type,
                "observation_type": observation_type
            }
            sightings.append(sighting)

    # JSONファイルに保存
    with open(json_path, 'w', encoding='utf-8') as f:
        json.dump(sightings, f, ensure_ascii=False, indent=2)

except Exception as e:
    print(f"エラーが発生しました: {str(e)}")

In [29]:
#山梨県のクマ目撃情報のPDFから情報を抽出してbear_sightings_yamanashiへ
import pdfplumber
import os
import json
import re

# PDFパスと出力JSONファイル名
pdf_path = 'kuma_r6_yamanashi.pdf'
json_path = os.path.join(os.getcwd(), "bear_sightings_yamanashi.json")

try:
    sightings = []
    all_lines = []
    
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text = page.extract_text()
            if text:
                lines = text.split('\n')
                all_lines.extend(lines)

    date_pattern = re.compile(r'(\d{4}/\d{1,2}/\d{1,2})')
    # 市区町村名を認識するパターン
    city_pattern = re.compile(r'(.+?[市町村])(.*)')
    # 地名ブロックを認識するパターン（「地内」や天候の前まで）
    location_pattern = re.compile(r'([^晴雨曇]{2,}?)((?:晴|雨|曇|霧|雪|地内).*)')

    for line in all_lines:
        # 不要行除外
        if not line.strip() or '《目撃・痕跡・その他》' in line:
            continue
            
        m = date_pattern.search(line)
        if not m:
            # 日付パターンが見つからない行はスキップ
            continue
            
        date_str = m.group(1)
        after_date_part = line[m.end():].strip()
        
        # "頃" の直後に空白がない場合は補間
        after_date_part = re.sub(r'頃(?!\s)', '頃 ', after_date_part)
        
        # 分割
        parts = after_date_part.split()
        
        # partsの最小要素数チェック
        if len(parts) < 3:
            print(f"Line parsing issue (not enough parts): {line}")
            continue
            
        # timeは最初の要素(例: "21:30頃")
        time = parts[0]
        
        # 残りの部分を結合して市区町村と地名を抽出
        remaining_text = ' '.join(parts[1:])
        city_match = city_pattern.match(remaining_text)
        
        if city_match:
            city = city_match.group(1)
            location_full = city_match.group(2).strip()
            
            # 地名部分を天候情報の前で分割
            location_match = location_pattern.match(location_full)
            if location_match:
                location = location_match.group(1).strip()
            else:
                # 天候情報等が見つからない場合は全体を使用
                location = location_full.split()[0] if location_full.split() else location_full
        else:
            # マッチしない場合は元の処理を使用
            city = parts[1]
            location = parts[2]
        
        # bear_countを抽出
        remaining = parts[3:]
        nums = [re.sub(r'\D', '', x) for x in remaining if re.search(r'\d+', x)]
        bear_count = nums[-1] if nums else "不明"
        
        sighting = {
            "date": date_str,
            "time": time,
            "city": city,
            "location": location,
            "bear_count": bear_count
        }
        sightings.append(sighting)

    # JSONファイルに保存
    with open(json_path, 'w', encoding='utf-8') as f:
        json.dump(sightings, f, ensure_ascii=False, indent=2)
    print(f"データをJSONファイルに保存しました: {json_path}")
    
except Exception as e:
    print(f"エラーが発生しました: {str(e)}")

データをJSONファイルに保存しました: /Users/nozomukitamura/Berkeley/bear_sightings_yamanashi.json


In [30]:
#静岡県のクマ目撃情報のPDFから情報を抽出してbear_sightings_shizuokaへ
import pdfplumber
import pandas as pd
import json
import re

def extract_text_from_regions(pdf_path, regions):
    """
    特定の座標範囲からテキストを抽出する関数
    
    Parameters:
    pdf_path (str): PDFファイルのパス
    regions (list): 抽出する領域のリスト。各領域は (x0, top, x1, bottom) のタプル
    """
    extracted_texts = []
    
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            for region in regions:
                # 指定された座標で領域を切り出し
                crop = page.crop(region)
                # テキストを抽出
                text = crop.extract_text()
                if text:
                    extracted_texts.append(text)
    
    return extracted_texts

def parse_bear_sightings(texts):
    """
    抽出したテキストから熊の目撃情報をパースする関数
    """
    sightings = []
    
    for text in texts:
        lines = text.split('\n')
        for line in lines:
            # 番号、目撃日、市町、地名のパターンに一致する行を抽出
            match = re.match(r'^(\d+(?:-\d+)?)\s+(\d+月\d+日)\s+(\S+)\s+(.+)$', line.strip())
            if match:
                sighting = {
                    "number": match.group(1),
                    "date": match.group(2),
                    "municipality": match.group(3),
                    "location": match.group(4).strip()
                }
                sightings.append(sighting)
    
    return sightings

def main():
    # 入力ファイル
    pdf_path = "241204kuma.pdf"
    
    # 抽出したい領域を定義（x0, top, x1, bottom）
    regions = [
        (30, 40, 120, 540),  # 領域1
        (125, 100, 200, 470)  # 領域2
    ]
    
    # テキストを抽出
    extracted_texts = extract_text_from_regions(pdf_path, regions)
    
    # 熊の目撃情報をパース
    sightings = parse_bear_sightings(extracted_texts)
    
    # DataFrameに変換
    df = pd.DataFrame(sightings)
    
    # JSONファイルに保存
    with open('bear_sightings_shizuoka.json', 'w', encoding='utf-8') as f:
        json.dump(sightings, f, ensure_ascii=False, indent=2)
    
    # 結果を表示
    print("抽出された目撃情報:")
    print(df.head())
    print(f"\n合計目撃件数: {len(df)}")

if __name__ == "__main__":
    main()

抽出された目撃情報:
  number   date municipality location
0      1   4月5日         富士宮市       佐折
1      2   4月7日         富士宮市       貫戸
2      3  4月11日         富士宮市       北山
3      4  4月30日         富士宮市      沼久保
4    5-1   5月1日         富士宮市       内房

合計目撃件数: 112


## ３種類のjsonを統合、データの型も揃える　緯度経度　情報の追加

In [34]:
import pandas as pd
import json
from datetime import datetime

def load_json_file(file_path: str):
    """指定したパスのJSONファイルをロードして返す。"""
    with open(file_path, 'r', encoding='utf-8') as f:
        return json.load(f)

def convert_date(date_str: str) -> pd.Timestamp:
    """
    例: '2024/6/4' や '6月19日' を pandas の日付型に変換。
    年が書かれていない場合は 2024 年を仮定しているサンプル。
    """
    if not date_str:
        return pd.NaT
    
    # スラッシュ区切りの場合 (例: '2024/6/4')
    if '/' in date_str:
        return pd.to_datetime(date_str, errors='coerce')
    
    # '6月19日' などの場合 (年が無いので 2024 年を付与する例)
    elif '月' in date_str and '日' in date_str:
        current_year = 2024
        # "6月19日" → "2024年6月19日" → "2024/6/19"
        date_str_mod = f"{current_year}年{date_str}"
        date_str_mod = date_str_mod.replace('年', '/').replace('月', '/').replace('日', '')
        return pd.to_datetime(date_str_mod, errors='coerce')
    
    # 上記以外の形式は NaT として扱う
    return pd.NaT

def parse_kanagawa_location(loc_str: str):
    """
    神奈川県のlocation文字列(例: '清川村煤ケ谷')から
    '市' '町' '村' のいずれかを見つけて分割する。
      - 見つかれば → city='清川村', location='煤ケ谷'
      - 見つからなければ city=NaN, location=loc_str
    """
    if not loc_str:  # None や空文字の場合
        return pd.NA, pd.NA
    
    boundary_words = ["市", "町", "村"]
    idx = None
    boundary_char = None
    
    # 最初に見つかった "市" "町" "村" を分割位置に
    for bw in boundary_words:
        i = loc_str.find(bw)
        if i != -1:
            # 最も手前(小さいインデックス)のものを優先
            if idx is None or i < idx:
                idx = i
                boundary_char = bw
    
    if idx is not None:
        # city 部分は "○○市(町/村)" まで含める
        city_str = loc_str[: idx + len(boundary_char)]
        # 残りを location として扱う
        loc_str_remain = loc_str[idx + len(boundary_char) :]
        loc_str_remain = loc_str_remain.strip()  # 前後の空白除去
        return city_str, loc_str_remain
    else:
        # "市/町/村" が見つからない場合
        return pd.NA, loc_str

def combine_json_data() -> pd.DataFrame:
    """
    3つのJSONファイル(神奈川県, 静岡県, 山梨県)を読み込み、
    カラムを揃えて1つのDataFrameにまとめた上で、日付順にソートして返す。
    """
    # 1) JSONファイルを読み込み 
    kanagawa_data = load_json_file('bear_sightings_kanagawa.json')
    shizuoka_data = load_json_file('bear_sightings_shizuoka.json')
    yamanashi_data = load_json_file('bear_sightings_yamanashi.json')
    
    # 万が一、戻り値が list でなかった場合に備えて list 化
    if not isinstance(kanagawa_data, list):
        kanagawa_data = [kanagawa_data]
    if not isinstance(shizuoka_data, list):
        shizuoka_data = [shizuoka_data]
    if not isinstance(yamanashi_data, list):
        yamanashi_data = [yamanashi_data]

    normalized_data = []
    
    # ==== 神奈川データ ====
    for rec in kanagawa_data:
        raw_location = rec.get('location')
        # 神奈川は 'location' に市町村 + それ以降の地名が合体している想定
        city_kanagawa, loc_kanagawa = parse_kanagawa_location(raw_location)
        
        normalized_data.append({
            'prefecture': '神奈川県',
            'date': rec.get('date'),
            'city': city_kanagawa,       # parse_kanagawa_location の結果
            'location': loc_kanagawa     # parse_kanagawa_location の結果
        })
    
    # ==== 静岡データ ====
    for rec in shizuoka_data:
        normalized_data.append({
            'prefecture': '静岡県',
            'date': rec.get('date'),
            'city': rec.get('municipality'),  # 静岡データは 'municipality' キー
            'location': rec.get('location')
        })
    
    # ==== 山梨データ ====
    for rec in yamanashi_data:
        normalized_data.append({
            'prefecture': '山梨県',
            'date': rec.get('date'),
            'city': rec.get('city'),          # 山梨データは 'city' キー
            'location': rec.get('location')
        })
    
    # 3) pandas DataFrame へ
    df = pd.DataFrame(normalized_data, columns=['prefecture', 'date', 'city', 'location'])
    
    # 4) 日付型変換
    df['date'] = df['date'].apply(convert_date)
    
    # 5) 日付順でソート
    df = df.sort_values('date', na_position='last').reset_index(drop=True)
    
    return df

# ------------------------
# Notebook (or Script) メイン処理
# ------------------------
if __name__ == "__main__":
    df_all = combine_json_data()
    
    print("==== 統合されたデータ ====")
    print(df_all.head())
    print(df_all.tail())
    print(df_all.info())
    
    # CSVに書き出し
    output_csv = 'bear_sightings_combined.csv'
    df_all.to_csv(output_csv, index=False, encoding='utf-8')
    print(f"DataFrameをCSVに出力しました: {output_csv}")


==== 統合されたデータ ====
  prefecture       date   city location
0        山梨県 2024-04-01    都留市       大野
1       神奈川県 2024-04-02    箱根町      宮城野
2        山梨県 2024-04-02  市川三郷町       黒沢
3        山梨県 2024-04-04  市川三郷町       黒沢
4        静岡県 2024-04-05   富士宮市       佐折
    prefecture       date    city location
529       神奈川県 2024-12-05     愛川町       半原
530        山梨県 2024-12-06     笛吹市   御坂町藤野木
531       神奈川県 2024-12-12     松田町     松田惣領
532        山梨県 2024-12-17  富士河口湖町       西湖
533        山梨県 2024-12-23    上野原市       鶴島
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 534 entries, 0 to 533
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   prefecture  534 non-null    object        
 1   date        534 non-null    datetime64[ns]
 2   city        526 non-null    object        
 3   location    530 non-null    object        
dtypes: datetime64[ns](1), object(3)
memory usage: 16.8+ KB
None
DataFrameをCSVに出力しました: bear

In [ ]:
import yaml
import pandas as pd
import numpy as np

############################
# 1. 郡名マッピングの定義
############################
CITY_GUN_MAP = {
    # (都道府県, 市町村名) : "郡名＋市町村名"

    # --- 神奈川県 ---
    ("神奈川県", "葉山町"): "三浦郡葉山町",
    ("神奈川県", "二宮町"): "中郡二宮町",
    ("神奈川県", "大磯町"): "中郡大磯町",
    ("神奈川県", "愛川町"): "愛甲郡愛川町",
    ("神奈川県", "清川村"): "愛甲郡清川村",
    ("神奈川県", "中井町"): "足柄上郡中井町",
    ("神奈川県", "大井町"): "足柄上郡大井町",
    ("神奈川県", "山北町"): "足柄上郡山北町",
    ("神奈川県", "松田町"): "足柄上郡松田町",
    ("神奈川県", "開成町"): "足柄上郡開成町",
    ("神奈川県", "湯河原町"): "足柄下郡湯河原町",
    ("神奈川県", "真鶴町"): "足柄下郡真鶴町",
    ("神奈川県", "箱根町"): "足柄下郡箱根町",
    ("神奈川県", "寒川町"): "高座郡寒川町",

    # --- 山梨県 ---
    ("山梨県", "昭和町"): "中巨摩郡昭和町",
    ("山梨県", "丹波山村"): "北都留郡丹波山村",
    ("山梨県", "小菅村"): "北都留郡小菅村",
    ("山梨県", "南部町"): "南巨摩郡南部町",
    ("山梨県", "富士川町"): "南巨摩郡富士川町",
    ("山梨県", "早川町"): "南巨摩郡早川町",
    ("山梨県", "身延町"): "南巨摩郡身延町",
    ("山梨県", "富士河口湖町"): "南都留郡富士河口湖町",
    ("山梨県", "山中湖村"): "南都留郡山中湖村",
    ("山梨県", "忍野村"): "南都留郡忍野村",
    ("山梨県", "西桂町"): "南都留郡西桂町",
    ("山梨県", "道志村"): "南都留郡道志村",
    ("山梨県", "鳴沢村"): "南都留郡鳴沢村",
    ("山梨県", "市川三郷町"): "西八代郡市川三郷町",

    # --- 静岡県 ---
    ("静岡県", "森町"): "周智郡森町",
    ("静岡県", "吉田町"): "榛原郡吉田町",
    ("静岡県", "川根本町"): "榛原郡川根本町",
    ("静岡県", "函南町"): "田方郡函南町",
    ("静岡県", "南伊豆町"): "賀茂郡南伊豆町",
    ("静岡県", "東伊豆町"): "賀茂郡東伊豆町",
    ("静岡県", "松崎町"): "賀茂郡松崎町",
    ("静岡県", "河津町"): "賀茂郡河津町",
    ("静岡県", "西伊豆町"): "賀茂郡西伊豆町",
    ("静岡県", "小山町"): "駿東郡小山町",
    ("静岡県", "清水町"): "駿東郡清水町",
    ("静岡県", "長泉町"): "駿東郡長泉町",
}

############################
# 2. 変換 & クリーニング関数
############################

def fix_city_name(pref: str, city: str) -> str:
    """
    「(都道府県, 市町村名) => 郡名+市町村名」の対応表で市町村名を置き換える。
    なければそのまま返す。
    """
    if pd.isna(city):
        return ""
    return CITY_GUN_MAP.get((pref, city), city)

def clean_address(city: str, location: str) -> tuple[str, str]:
    """
    city, location の文字列をクレンジングして新しい (city, location) を返す。
    
    例:
    - 「緑区」を city に合体
    - 「・」が入っていれば手前だけ採用
    - 括弧内の文字列を除去
    - 不要な文字列を除去
    """
    # NaN→空文字
    city = '' if pd.isna(city) else str(city)
    location = '' if pd.isna(location) else str(location)

    # 市名の重複を削除 (あれば)
    if city and location.startswith(city):
        location = location[len(city):].strip()

    # 「緑区」が location にあれば city に合体
    if "緑区" in location and "緑区" not in city:
        city += "緑区"
        location = location.replace("緑区", "")

    # 「・」で分割 → 先頭だけ
    if "・" in location:
        location = location.split("・")[0]

    # 正規表現で括弧内を削除（全角＆半角）
    location = re.sub(r'（.*?）', '', location)  # 全角括弧
    location = re.sub(r'\(.*?\)', '', location)  # 半角括弧

    # 不要単語を置換
    remove_words = ["付近", "峠", "地区", "地内", "山地", "徳間", "鯨野", "釜の口", "諏訪内", "大道", "佐野区"]
    for word in remove_words:
        location = location.replace(word, '')

    return city.strip(), location.strip()

############################
# 3. YAML のロード
############################

def load_geo_cache(yaml_path='areas_with_coords.yml') -> dict:
    """
    事前に作ったYAMLキャッシュ(都道府県→市町村→町域→{longitude,latitude})を読み込み
    """
    with open(yaml_path, 'r', encoding='utf-8') as f:
        return yaml.safe_load(f)

############################
# 4. 座標検索 with フォールバック
############################

def lookup_coords(pref: str, city: str, area: str, geo_dict: dict) -> dict:
    """
    geo_dict[pref][city][area] の座標を返す。
    見つからなければ、同じ city 内の "以下に掲載がない場合" を試し、
    それもなければ {'longitude':None, 'latitude':None}.
    """
    try:
        return geo_dict[pref][city][area]
    except KeyError:
        # (1) area が見つからない場合 → "以下に掲載がない場合" を試す
        try:
            return geo_dict[pref][city]["以下に掲載がない場合"]
        except KeyError:
            # (2) それでもなければ None
            return {"longitude": None, "latitude": None}

############################
# 5. DataFrameに座標を付与
############################

def add_coords_from_cache(df: pd.DataFrame, geo_dict: dict) -> pd.DataFrame:
    """
    dfに対して 'prefecture','city','location' 列からジオコーディングし、
    'longitude','latitude' 列を追加して返す。
    """
    longitudes = []
    latitudes = []
    
    for _, row in df.iterrows():
        # 1) 元データから取得
        pref = row.get('prefecture')
        city_raw = row.get('city')
        loc_raw = row.get('location')
        
        # 2) city / location をクリーニング
        city_cleaned, loc_cleaned = clean_address(city_raw, loc_raw)
        
        # 3) 郡名付きの正式名称に変換
        city_fixed = fix_city_name(pref, city_cleaned)
        
        # 4) YAML辞書から座標を探す (フォールバックで "以下に掲載がない場合" も試す)
        coords = lookup_coords(pref, city_fixed, loc_cleaned, geo_cache)
        
        longitudes.append(coords['longitude'])
        latitudes.append(coords['latitude'])
    
    df['longitude'] = longitudes
    df['latitude'] = latitudes
    return df

############################
# 6. メイン処理フロー
############################

if __name__ == "__main__":
    # 1) CSV 読込
    df_bears = pd.read_csv('bear_sightings_combined.csv', encoding='utf-8')

    # 2) YAML キャッシュ読込
    geo_cache = load_geo_cache('areas_with_coords.yml')

    # 3) 座標付与
    df_bears_with_coords = add_coords_from_cache(df_bears, geo_cache)

    # 4) 結果保存 & 確認
    df_bears_with_coords.to_csv('bear_sightings_with_coords.csv', index=False, encoding='utf-8')


==== 先頭10行を表示 ====
  prefecture        date    city location   longitude   latitude
0        山梨県  2024-04-01     都留市       大野  138.942398  35.516033
1       神奈川県  2024-04-02     箱根町      宮城野  139.048584  35.263847
2        山梨県  2024-04-02   市川三郷町       黒沢  138.466721  35.531780
3        山梨県  2024-04-04   市川三郷町       黒沢  138.466721  35.531780
4        静岡県  2024-04-05    富士宮市       佐折  138.552734  35.325047
5        静岡県  2024-04-07    富士宮市       貫戸  138.620285  35.193417
6        静岡県  2024-04-11    富士宮市       北山  138.641251  35.315876
7        山梨県  2024-04-11  富士河口湖町       河口  138.779053  35.543289
8       神奈川県  2024-04-11    相模原市    緑区佐野川  139.141830  35.651836
9       神奈川県  2024-04-12     松田町        寄  139.124084  35.405666


In [ ]:
# 例: 既存の結果ファイルを読み込む
df = pd.read_csv('bear_sightings_with_coords.csv', encoding='utf-8')

# longitude または latitude が NaN になっている行数
num_na_long = df['longitude'].isna().sum()
num_na_lat = df['latitude'].isna().sum()

print(f"longitude が NaN の行数: {num_na_long}")
print(f"latitude が NaN の行数:  {num_na_lat}")

# 全行数に対してどの程度かざっくり確認
print(f"全体行数: {len(df)}")

==== クリーニング前 ====
longitude が NaN の行数: 8
latitude が NaN の行数:  8
全体行数: 534


## GISの実験中

In [ ]:
#　参考　ヒートマップ機能
import pandas as pd
import folium
from datetime import datetime, timedelta
from folium import plugins

# 基本的な地図を作成（日本全体が見えるように）
m = folium.Map(
    location=[38.0, 138.0],  # 日本のほぼ中心
    zoom_start=5,
    tiles='CartoDB positron'
)

try:
    # CSVファイルの読み込み
    df = pd.read_csv('bear_sightings_with_coords.csv')
    
    # NaNを含む行を削除
    df = df.dropna(subset=['latitude', 'longitude'])
    
    # 日付を datetime に変換
    df['date'] = pd.to_datetime(df['date'], errors='coerce')
    df = df.dropna(subset=['date'])  # 無効な日付を削除
    
    # 最近1週間のデータを判定
    one_week_ago = datetime.now() - timedelta(days=7)
    df['is_recent'] = df['date'] >= one_week_ago

    # マーカークラスターの作成
    marker_cluster = plugins.MarkerCluster(name='目撃地点（クラスター）')

    # データが存在する場合、マーカーを追加
    for _, row in df.iterrows():
        # 最近の目撃は赤、それ以外は青で表示
        color = 'red' if row['is_recent'] else 'blue'
        radius = 8 if row['is_recent'] else 6
        
        # 日時の文字列を作成（エラー処理付き）
        try:
            date_str = row['date'].strftime('%Y-%m-%d %H:%M')
        except:
            date_str = str(row['date'])
        
        folium.CircleMarker(
            location=[row['latitude'], row['longitude']],
            radius=radius,
            color=color,
            fill=True,
            fill_opacity=0.7,
            popup=f"日時: {date_str}"
        ).add_to(marker_cluster)

    # クラスターを地図に追加
    marker_cluster.add_to(m)

    # 有効なデータポイントのみでヒートマップを作成
    heat_data = df[['latitude', 'longitude']].values.tolist()
    plugins.HeatMap(heat_data, name='熊出没密度').add_to(m)

    # レイヤーコントロールの追加
    folium.LayerControl().add_to(m)

    # 凡例の追加
    legend_html = '''
    <div style="position: fixed; 
                bottom: 50px; right: 50px; 
                border: 2px solid grey; 
                background-color: white;
                padding: 10px;">
        <p><strong>目撃情報</strong></p>
        <p>
            <span style="color: red;">●</span> 過去1週間<br>
            <span style="color: blue;">●</span> それ以前
        </p>
    </div>
    '''
    m.get_root().html.add_child(folium.Element(legend_html))

    # 統計情報の追加（除外されたデータの情報も含める）
    total_original = len(pd.read_csv('bear_sightings_with_coords.csv'))
    valid_points = len(df)
    recent_points = df['is_recent'].sum()
    
    stats_html = f'''
    <div style="position: fixed; 
                bottom: 50px; left: 50px; 
                border: 2px solid grey; 
                background-color: white;
                padding: 10px;">
        <p><strong>統計情報</strong></p>
        <p>
            有効データ: {valid_points}件<br>
            過去1週間: {recent_points}件<br>
            <span style="color: #666; font-size: 0.9em;">
                （無効データ: {total_original - valid_points}件）
            </span>
        </p>
    </div>
    '''
    m.get_root().html.add_child(folium.Element(stats_html))

except FileNotFoundError:
    print("CSVファイルが見つかりません")
except Exception as e:
    print(f"エラーが発生しました: {str(e)}")

# 地図を表示
m

In [150]:
import pandas as pd
import folium
from datetime import datetime, timedelta
from folium import plugins
from collections import defaultdict
import json

# 基本的な地図を作成
m = folium.Map(
    location=[35.5, 138.5],  # 山梨県付近を中心に
    zoom_start=8,
    tiles='CartoDB positron',
    control_scale=True
)

try:
    # CSVファイルの読み込み
    df = pd.read_csv('bear_sightings_with_coords.csv')
    df = df.dropna(subset=['latitude', 'longitude'])
    
    # 日付を datetime に変換
    df['date'] = pd.to_datetime(df['date'], errors='coerce')
    df = df.dropna(subset=['date'])
    
    # 最近1週間のデータを判定
    one_week_ago = datetime.now() - timedelta(days=7)
    df['is_recent'] = df['date'] >= one_week_ago

    # 市町村別の集計
    city_counts = df['city'].value_counts()
    recent_city_counts = df[df['is_recent']]['city'].value_counts()

    # 時系列データの準備（月別集計）
    df['month'] = df['date'].dt.strftime('%Y-%m')
    monthly_counts = df['month'].value_counts().sort_index()

    # レイヤーの作成
    railway_layer = folium.FeatureGroup(name='JR路線', show=True)
    recent_layer = folium.FeatureGroup(name='過去1週間の目撃情報', show=True)
    old_layer = folium.FeatureGroup(name='過去の目撃情報', show=True)

    # 鉄道路線データの定義
    # 東海道線のデータ
    tokaido_line = {
        "type": "Feature",
        "properties": {"name": "東海道線"},
        "geometry": {
            "type": "LineString",
            "coordinates": [
                [138.3833, 34.9757],  # 熱海
                [138.7833, 35.1027],  # 沼津
                [138.8889, 35.1258],  # 三島
                [138.9889, 35.1617],  # 富士
                [139.0889, 35.1847],  # 静岡
                [139.2889, 35.2147],  # 掛川
                [139.3889, 35.2447],  # 浜松
            ]
        }
    }

    # 御殿場線のデータ
    gotemba_line = {
        "type": "Feature",
        "properties": {"name": "御殿場線"},
        "geometry": {
            "type": "LineString",
            "coordinates": [
                [138.7833, 35.1027],  # 沼津
                [138.8500, 35.2000],  # 御殿場
                [139.0500, 35.3000],  # 山北
                [139.1500, 35.3500],  # 松田
            ]
        }
    }

    # 身延線のデータ
    minobu_line = {
        "type": "Feature",
        "properties": {"name": "身延線"},
        "geometry": {
            "type": "LineString",
            "coordinates": [
                [138.8889, 35.1258],  # 富士
                [138.5167, 35.4667],  # 身延
                [138.5683, 35.6617],  # 甲府
            ]
        }
    }

    # 路線のスタイル設定
    style_function = lambda x: {
        'color': '#FF6B6B',
        'weight': 3,
        'opacity': 0.8,
        'dashArray': '5, 10'
    }

    # 路線を地図に追加
    folium.GeoJson(
        tokaido_line,
        name='東海道線',
        style_function=style_function,
        tooltip=folium.GeoJsonTooltip(fields=['name'])
    ).add_to(railway_layer)

    folium.GeoJson(
        gotemba_line,
        name='御殿場線',
        style_function=style_function,
        tooltip=folium.GeoJsonTooltip(fields=['name'])
    ).add_to(railway_layer)

    folium.GeoJson(
        minobu_line,
        name='身延線',
        style_function=style_function,
        tooltip=folium.GeoJsonTooltip(fields=['name'])
    ).add_to(railway_layer)

    # 市町村名のリストを作成
    city_options = ''.join([f'<option value="{city}">{city}</option>' 
                           for city in sorted(df['city'].unique())])

    # 月別グラフのバーを作成
    monthly_bars = ''.join([
        f'<div style="display: inline-block; width: {100/len(monthly_counts)}%; '
        f'background-color: #4a9eff; margin-right: 1px; '
        f'height: {count/monthly_counts.max()*100}%;" '
        f'title="{month}: {count}件"></div>'
        for month, count in monthly_counts.items()
    ])

    # 市町村別テーブルの行を作成
    city_rows = ''.join([
        f"<tr><td style='padding: 5px;'>{city}</td>"
        f"<td style='padding: 5px; text-align: right;'>{count}</td>"
        f"<td style='padding: 5px; text-align: right;'>{recent_city_counts.get(city, 0)}</td></tr>"
        for city, count in city_counts.items()
    ])

    # データポイントの追加
    for _, row in df.iterrows():
        color = '#dc2626' if row['is_recent'] else '#1d4ed8'
        radius = 8 if row['is_recent'] else 6
        
        location_info = row.get('location', '不明')
        city_info = row.get('city', '不明')
        
        popup_text = f"""
        <div style='min-width: 200px; font-family: Arial;'>
            <h4 style='margin-bottom: 10px; color: {color};'>熊の目撃情報</h4>
            <table style='width: 100%;'>
                <tr><td><strong>発生日時:</strong></td><td>{row['date'].strftime('%Y-%m-%d %H:%M')}</td></tr>
                <tr><td><strong>市町村:</strong></td><td>{city_info}</td></tr>
                <tr><td><strong>地点:</strong></td><td>{location_info}</td></tr>
                <tr><td><strong>緯度:</strong></td><td>{row['latitude']:.6f}</td></tr>
                <tr><td><strong>経度:</strong></td><td>{row['longitude']:.6f}</td></tr>
            </table>
        </div>
        """
        
        marker = folium.CircleMarker(
            location=[row['latitude'], row['longitude']],
            radius=radius,
            color=color,
            fill=True,
            fill_opacity=0.7,
            popup=popup_text,
            tooltip=f"{city_info} ({row['date'].strftime('%Y-%m-%d')})"
        )
        
        if row['is_recent']:
            marker.add_to(recent_layer)
        else:
            marker.add_to(old_layer)

    # レイヤーを地図に追加（鉄道レイヤーを最初に追加して他のマーカーの下に表示）
    railway_layer.add_to(m)
    recent_layer.add_to(m)
    old_layer.add_to(m)

    # 分析パネルのHTML
    # JavaScriptをPython文字列として適切にエスケープ
    js_code = """
        <script>
        (function() {
            var citySelect = document.getElementById('citySelect');
            if (citySelect) {
                citySelect.addEventListener('change', function() {
                    var selectedCity = this.value;
                    var markers = document.getElementsByClassName('leaflet-marker-icon');
                    
                    for (var i = 0; i < markers.length; i++) {
                        var marker = markers[i];
                        if (!selectedCity || marker.title.includes(selectedCity)) {
                            marker.style.display = 'block';
                        } else {
                            marker.style.display = 'none';
                        }
                    }
                });
            }
        })();
        </script>
    """

    analysis_html = f"""
    <div style="position: fixed; 
                top: 10px; right: 10px;
                width: 300px;
                background-color: white;
                border: 2px solid grey;
                padding: 10px;
                font-family: Arial;
                max-height: 80vh;
                overflow-y: auto;">
        
        <div style="margin-bottom: 15px;">
            <h4 style="margin: 0 0 8px 0;">市町村検索</h4>
            <select id="citySelect" style="width: 100%; padding: 5px;">
                <option value="">全ての市町村</option>
                {city_options}
            </select>
        </div>

        <div style="margin-bottom: 15px;">
            <h4 style="margin: 0 0 8px 0;">月別目撃件数</h4>
            <div style="height: 150px; display: flex; align-items: flex-end;">
                {monthly_bars}
            </div>
            <div style="font-size: 0.8em; margin-top: 5px; color: #666;">
                ※ グラフにマウスを重ねると詳細表示
            </div>
        </div>

        <div>
            <h4 style="margin: 0 0 8px 0;">市町村別目撃件数</h4>
            <div style="max-height: 200px; overflow-y: auto;">
                <table style="width: 100%; border-collapse: collapse;">
                    <tr style="background-color: #f3f4f6;">
                        <th style="padding: 5px; text-align: left;">市町村</th>
                        <th style="padding: 5px; text-align: right;">総数</th>
                        <th style="padding: 5px; text-align: right;">直近</th>
                    </tr>
                    {city_rows}
                </table>
            </div>
        </div>
    </div>
    {js_code}
    """

    # 分析パネルを地図に追加
    m.get_root().html.add_child(folium.Element(analysis_html))

    # レイヤーコントロールを追加
    folium.LayerControl(collapsed=False).add_to(m)
    
    # 全画面表示ボタンを追加
    plugins.Fullscreen(
        position='topleft',
        title='全画面表示',
        title_cancel='全画面解除',
        force_separate_button=True
    ).add_to(m)
    
    # ミニマップを追加
    minimap = plugins.MiniMap(toggle_display=True, position='bottomright')
    m.add_child(minimap)

except FileNotFoundError:
    print("CSVファイルが見つかりません")
except Exception as e:
    print(f"エラーが発生しました: {str(e)}")

# Jupyter Notebookで地図を表示
m